## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio



In [2]:
load_dotenv(override=True)

True

In [3]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("lchanio@echyperion.com")  # Change to your verified sender
    to_email = To("lchanio@echyperion.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "[SendGrid-dev] Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [4]:
# Add information about the product in the system prompt. This will be used to write the e-mails.
with open('ComplAI_brochure.md','r') as file:
    brochure_text = file.read()

instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails. Here is some information about the product:\n" + brochure_text

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response. Here is some information about the product:\n" + brochure_text

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails. Here is some information about the product:\n" + brochure_text

sales_agent_instructions = [instructions1, instructions2, instructions3] # we can iterate over the list later on
sales_agent_names = ["Professional Sales Agent", "Humorous Sales Agent", "Busy Sales Agent"]
sales_agents_metadata = zip(sales_agent_names, sales_agent_instructions)

In [5]:
sales_agents = []
for agent_name, instruction in list(sales_agents_metadata):
        sales_agents.append(Agent(
                name=agent_name,
                instructions=instruction,
                model="gpt-4o-mini"
        ))

print(f"Initializsed {len(sales_agents)} agents")

Initializsed 3 agents


### Read e-mails from a contact list

Let's update the agents to create e-mails for specific people from a contact list.
We will provide more context to the workflow in the form of a product brochure and recipients.
The agents will craft a personalized e-mail according to each recipient role. 



In [6]:
import pandas as pd

contacts = pd.read_csv('contact_list.csv')
print(contacts)


             Name                  Email                       Title
0      Emily Chen  proklos+sg1@gmail.com           Marketing Manager
1       David Lee  proklos+sg2@gmail.com              Hiring Manager
2    Sophia Patel  proklos+sg3@gmail.com  Digital Product Specialist
3     Jack Harris  proklos+sg4@gmail.com        Award-Winning Writer
4   Olivia Brooks  proklos+sg5@gmail.com           Data Science Lead
5     Thomas Reed  proklos+sg6@gmail.com     Chief Executive Officer
6  Adriana Garcia  proklos+sg7@gmail.com    Chief Compliance Officer


In [ ]:
# Test it
for keys,contact in contacts.iloc[5:7,:].iterrows(): # try it with the two C-level officers in the list
    contact_name = contact[['Name']]
    contact_title = contact['Title']
    for sales_agent in sales_agents:
        result = Runner.run_streamed(sales_agent, input=f"Write a cold sales email addressed to {contact_name} who is a {contact_title}. Adjust the content according to their position in their company.")
        async for event in result.stream_events():
            if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
                print(event.data.delta, end="", flush=True)
        print("-"*30)

## Parallel execution wrapper functions
1. Wrapped agent execution in an async function that collects the full streamed response
2. Used asyncio.gather(*tasks) to run all agents in parallel for each contact
3. Maintained streaming within each agent while parallelizing across agents
4. Preserved tracing structure
5. Coroutines to handle parallel execution for a single contact, or fully parallel (all agents work in parallel at all contacts)

Many thanks to Claude for helping out

In [14]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email number only.",
    model="gpt-4o-mini"
)

In [ ]:
async def run_agent_for_contact(sales_agent, contact_name, contact_title):
    """Run a single agent for a contact and collect the full response
    
    Args:
        sales_agent: The agent object that will generate the sales email
        contact_name (str): The name of the contact to address the email to
        contact_title (str): The job title of the contact
        
    Returns:
        str: The complete generated sales email text
        
    The function streams the agent's response and accumulates the text deltas
    into a single string containing the full email content.
    """
    full_response = ""
    result = Runner.run_streamed(sales_agent, input=f"Write a cold sales email addressed to {contact_name} who is a {contact_title}. Adjust the content according to their position in their company.")
    
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            full_response += event.data.delta
    
    return full_response

async def process_contact_parallel(contact, sales_agents):
    """Process a single contact with all agents in parallel
    Args:
        contact (pd.DataFrame): A DataFrame containing contact information.
        sales_agents (list): A list of Agent objects representing sales agents.
    Returns:
        list: A list of results from running the agents for the contact.
    """    
    
    contact_name = contact['Name']
    contact_title = contact['Title']
    
    # Create tasks for all agents for this contact
    tasks = [
        run_agent_for_contact(agent, contact_name, contact_title) 
        for agent in sales_agents
    ]
    
    # Run all agents in parallel for this contact
    with trace(f"Parallel cold emails for {contact_name}"):
        results = await asyncio.gather(*tasks)
    
    return results

async def execute_fully_parallel(contacts, sales_agents):
    """ Execute all e-mail composition agents in parallel 
    (3 agents per contact for each contact) 
    
    Args:
        contacts (pd.DataFrame): A DataFrame containing contact information.
        sales_agents (list): A list of Agent objects representing sales agents.
    Returns:
        None
    """

    all_tasks = []
    
    for keys, contact in contacts.iloc[5:7, :].iterrows():
        contact_name = contact['Name']
        contact_title = contact['Title']
        
        for agent in sales_agents:
            task = run_agent_for_contact(agent, contact_name, contact_title)
            all_tasks.append((contact_name, agent, task))
    
    # Run everything in parallel
    with trace("All parallel cold emails"):
        results = await asyncio.gather(*[task for _, _, task in all_tasks])
    
    # Process results
    for i, (contact_name, agent, _) in enumerate(all_tasks):
        print(f"\n{contact_name} - Agent {agent.name}: {results[i]}")


async def select_best_email(emails, picker_agent):
    email_txt = "These are the cold sales e-mails:\n".join(emails)

    best = await Runner.run(picker_agent, email_txt)
    # print(f"Best sales email:\n{best.final_output}")
    return best.final_output

# Execute one customer at a time with parallel composition agents
async def execute(contacts, sales_agents):
    """Execute email generation for each contact using multiple sales agents in parallel
    
    Args:
        contacts (pd.DataFrame): DataFrame containing contact information including Name and Title
        sales_agents (list): List of Agent objects that will generate sales emails
        
    Returns:
        pd.DataFrame: The contacts DataFrame with a new Email_Text column containing the best 
                     generated email for each contact
                     
    For each contact, this function:
    1. Generates multiple email versions in parallel using the sales agents
    2. Selects the best email using a picker agent
    3. Stores the selected email in the contacts DataFrame
    """
    for index, contact in contacts.iterrows():
        print(f"\nProcessing {contact['Name']}...")
        results = await process_contact_parallel(contact, sales_agents)
        print(f"Prepared {len(results)} e-mails for {contact['Name']}")
        emails=""
        # Print results from all agents
        for i, result in enumerate(results):
            emails += f"\n--- Agent {i+1} E-mail ---\n {result}"
        #print(emails)
        best = await select_best_email(emails, sales_picker)
        print(f"Selected: {best}")
        best_email = results[int(best)-1]
        contacts.at[index,"Email_Text"] = best_email
    
    return contacts


In [16]:
# Create a new agent - will pick contacts from the contact list that may be interested based on their positions.

customer_picker_instructions = f""" You are a sales manager assistant bot. 
You will examine the name and positions of potential recipients for a cold email campaign and decide if they are a good fit to receive the e-mail.
Respond only with YES or NO, without any preamble or explanation.
The following additional context about the product being marketed in the campaign will help you make a decision:
{brochure_text}
"""

customer_picker_agent = Agent(
        name="Customer Picker Agent",
        instructions=customer_picker_instructions,
        model="gpt-4o-mini",
)

contacts['Include'] = "-" # Initialize column indicating inclusion in the campaign.

for index,contact in contacts.iterrows():
    customer_name = contact['Name']
    customer_title = contact['Title']
    message = f"Is {customer_name} - {customer_title} of the company a good fit for the cold e-mail campaign? Respond with YES or NO only"
    result = Runner.run_streamed(
        customer_picker_agent, 
        input = message
    )
    full_response = ""
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            full_response += event.data.delta
    contacts.at[index, 'Include'] = full_response


campaign_contacts = contacts[contacts['Include']=='YES']

In [17]:
print(campaign_contacts)

             Name                  Email                       Title Include
2    Sophia Patel  proklos+sg3@gmail.com  Digital Product Specialist     YES
5     Thomas Reed  proklos+sg6@gmail.com     Chief Executive Officer     YES
6  Adriana Garcia  proklos+sg7@gmail.com    Chief Compliance Officer     YES


In [ ]:
# for Jupyter notebooks:
import nest_asyncio
nest_asyncio.apply()

# asyncio.run(execute_fully_parallel(campaign_contacts, sales_agents))
# not using the full parallelization at the moment as it requires extra effort 
# to match e-mails to recepients for selection in the next step. 

campaign = asyncio.run(execute(campaign_contacts, sales_agents))
print(campaign)


Processing Sophia Patel...
Prepared 3 e-mails for Sophia Patel
Selected: 2

Processing Thomas Reed...


/tmp/ipykernel_715550/3461193765.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contacts.at[index,"Email_Text"] = best_email


Prepared 3 e-mails for Thomas Reed
Selected: 2

Processing Adriana Garcia...
Prepared 3 e-mails for Adriana Garcia
Selected: 2
             Name                  Email                       Title Include  \
2    Sophia Patel  proklos+sg3@gmail.com  Digital Product Specialist     YES   
5     Thomas Reed  proklos+sg6@gmail.com     Chief Executive Officer     YES   
6  Adriana Garcia  proklos+sg7@gmail.com    Chief Compliance Officer     YES   

                                          Email_Text  
2  Subject: Let’s Make SOC 2 Compliance Your New ...  
5  Subject: Let’s Turn Compliance Into Your Secre...  
6  Subject: Elevate Your Compliance Game—Without ...  


In [25]:
print(results)
#outputs = [result.final_output for result in results]

#for output in outputs:
#    print(output + "\n\n")


['Subject: Elevate Compliance Efficiency with AI-Driven Solutions\n\nDear Adriana,\n\nAs Chief Compliance Officer, you understand the critical role that seamless audit processes and ongoing compliance play in an organization’s success. At ComplAI, we believe compliance doesn’t have to be painful. Our AI-powered SaaS platform transforms the way companies prepare for SOC 2 audits, offering continuous compliance that fits the demands of today’s dynamic environments.\n\nHere’s how ComplAI can enhance your compliance framework:\n\n- **AI-Driven Readiness**: Our system scans policies and workflows in real-time, identifying gaps before they become issues during audits. This proactive approach minimizes last-minute scrambles.\n\n- **Zero-Stress Audit Preparation**: With one-click evidence packaging and custom auditor-friendly reports, your team can focus on what truly matters—ensuring compliance instead of managing chaos.\n\n- **Continuous Compliance**: We help you stay compliant every day, no

In [ ]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [ ]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [ ]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [ ]:
sales_agent1

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [ ]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("lchanio@echyperion.com")  # Change to your verified sender
    to_email = To("proklos+sg@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [ ]:
# Let's look at it
send_email

### And you can also convert an Agent into a tool

In [ ]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [ ]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

## And now it's time for our Sales Manager - our planning agent

In [ ]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [ ]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("lchanio@echyperion.com")  # Change to your verified sender
    to_email = To("proklos+sg@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
tools = [subject_tool, html_tool, send_html_email]

In [ ]:
tools

In [ ]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


### Now we have 3 tools and 1 handoff

In [ ]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

In [ ]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.